In [ ]:
!pip install --quiet imbalanced-learn==0.10.1

# V3

In [ ]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (AIMRAS)
# VERSION: 11.0 (Advanced Modeling & Hyperparameter Tuning Edition)
#
# KEY ENHANCEMENTS FROM PREVIOUS VERSION:
# 1. ADVANCED ENSEMBLE MODEL: Replaced the VotingClassifier with a more powerful
#    StackingClassifier, which uses a meta-model to learn the best way to
#    combine predictions from base models.
# 2. HYPERPARAMETER TUNING: Integrated GridSearchCV to automatically find the
#    optimal parameters for the RandomForest component of the ensemble,
#    improving model accuracy and robustness.
# 3. ENHANCED FEATURE ENGINEERING: Added a 'temperature_x_humidity' interaction
#    term to better capture the combined effects critical for microbial risk.
# 4. SHAP PLOT FIX: Resolved the IndexError by using the modern shap.plots.bar()
#    function correctly for multi-class outputs.
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# Core package installation with a known-stable set of pinned versions.
# !pip install --quiet \
#     numpy==1.24.3 \
#     pandas==2.0.3 \
#     scikit-learn==1.3.2 \
#     imbalanced-learn==0.11.0 \
#     scipy==1.11.4 \
#     xgboost==2.0.3 \
#     shap==0.45.1 \
#     seaborn==0.13.2 \
#     matplotlib==3.8.4 \
#     joblib==1.4.2

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings
import json
import os
import re
import traceback
from typing import Dict, List, Tuple, Any

# Scikit-learn, Imblearn, XGBoost
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_curve, roc_curve, auc
)
from sklearn.ensemble import IsolationForest, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# Silence non-critical warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)

# ------------------------------------------------------------------------------
# SECTION B: CONFIGURATION SYSTEM
# ------------------------------------------------------------------------------

class AIMRASConfig:
    """Simplified configuration management for the AIMRAS analytics script."""
    DEFAULT_CONFIG = {
        "data": {
            "historical_file_path": "/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv",
        },
        "processing": {
            "anomaly_contamination": 0.01,
        },
        "model": {
            "train_test_split": 0.8,
            "output_dir": "aimras_outputs" # Directory for saved models and graphs
        },
        "microbial_thresholds": {
            "mold": {
                "rh_sustained": 80.0,  # % RH
                "rh_duration": 48,     # hours
                "temperature_range": [5, 35],  # °C
            },
            "virus_persistence": {
                "rh_low_risk": 20, # % RH below this increases persistence
                "rh_high_risk": 80, # % RH above this increases persistence
                "co2_ventilation_proxy": 1000 # ppm, high CO2 indicates poor ventilation
            }
        }
    }

    def __init__(self, config_file=None):
        self.config = self.DEFAULT_CONFIG.copy()
        if config_file and os.path.exists(config_file):
            with open(config_file, 'r') as f:
                file_config = json.load(f)
                self._deep_update(self.config, file_config)
                print(f"Loaded configuration from {config_file}")
        print("Configuration initialized.")

    def _deep_update(self, d, u):
        for k, v in u.items():
            if isinstance(v, dict) and k in d and isinstance(d[k], dict):
                self._deep_update(d[k], v)
            else:
                d[k] = v

    def get(self, section, key=None):
        if key is None:
            return self.config.get(section, {})
        return self.config.get(section, {}).get(key)

# ------------------------------------------------------------------------------
# SECTION C: CORE ANALYTICS ENGINE
# ------------------------------------------------------------------------------

class EnvironmentalDiagnosticEngine:
    """
    A comprehensive engine for indoor air quality analysis, featuring advanced
    feature engineering, microbial risk assessment, and ensemble modeling.
    """
    def __init__(self, config_file=None):
        self.config = AIMRASConfig(config_file)
        self.df = None
        self.X_train, self.X_test = None, None
        self.y_train, self.y_test = None, None
        self.models = {}
        self.scaler = None
        self.encoder = None
        self.feature_names = None
        self.report_data = {} # To store data for the final report
        self.output_dir = self.config.get("model", "output_dir")
        os.makedirs(self.output_dir, exist_ok=True)
        print(f"✅ EnvironmentalDiagnosticEngine initialized. Outputs will be saved to '{self.output_dir}/'")

    def _load_and_clean_data(self, filepath: str):
        """Loads and cleans the dataset with robust validation and processing."""
        print("--> STAGE 1: Loading and Cleaning Data...")
        
        if not os.path.exists(filepath):
            error_message = (
                f"FATAL ERROR: Dataset not found at {filepath}\n\n"
                "--- How to fix on Kaggle: ---\n"
                "1. Click '+ Add data' in the right-hand pane.\n"
                "2. Search for 'IoT Indoor Air Quality Dataset' and add it."
            )
            raise FileNotFoundError(error_message)
        
        try:
            self.df = pd.read_csv(filepath)
            print(f"Loaded {len(self.df)} rows from {filepath}")

            def clean_col_name(column_name):
                name = re.sub(r'\(.*\)', '', column_name)
                name = name.strip().lower().replace(' ', '_').strip('_')
                return name

            self.df.rename(columns=clean_col_name, inplace=True)
            print(f"Standardized column names: {self.df.columns.tolist()}")
            
            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'], errors='coerce')
            self.df = self.df.dropna(subset=['timestamp']).set_index('timestamp').sort_index()
            self.df = self.df[~self.df.index.duplicated(keep='last')]

            numeric_cols = self.df.select_dtypes(include=np.number).columns
            self.df[numeric_cols] = self.df[numeric_cols].interpolate(method='time')
            self.df = self.df.ffill().bfill()

            for col in numeric_cols:
                Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower, upper = Q1 - 2.5 * IQR, Q3 + 2.5 * IQR
                if ((self.df[col] < lower) | (self.df[col] > upper)).any():
                    print(f"Capping outliers in '{col}'.")
                    self.df[col] = self.df[col].clip(lower, upper)

            print(f"Data cleaned successfully. Final shape: {self.df.shape}")

        except Exception as e:
            print(f"ERROR during data loading/cleaning: {e}")
            raise

    def _generate_base_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Generates all features for a given dataframe, callable by train and predict."""
        df_out = df.copy()

        # Time-based features
        df_out['hour'] = df_out.index.hour
        df_out['day_of_week'] = df_out.index.dayofweek
        df_out['is_weekend'] = df_out['day_of_week'].isin([5, 6]).astype(int)
        df_out['hour_sin'] = np.sin(2 * np.pi * df_out['hour'] / 24)
        df_out['hour_cos'] = np.cos(2 * np.pi * df_out['hour'] / 24)

        # Rolling statistics
        for col in ['temperature', 'humidity', 'co2', 'pm2.5']:
            if col in df_out.columns:
                df_out[f'{col}_24h_mean'] = df_out[col].rolling(24, min_periods=1).mean()
                df_out[f'{col}_24h_std'] = df_out[col].rolling(24, min_periods=1).std()
                df_out[f'{col}_rate_of_change'] = df_out[col].diff()

        # Microbial/physical features
        mold_cfg = self.config.get("microbial_thresholds", "mold")
        df_out['temp_x_humidity'] = df_out['temperature'] * df_out['humidity']
        is_wet = (df_out['humidity'] > mold_cfg["rh_sustained"]).astype(int)
        df_out[f'time_of_wetness_{mold_cfg["rh_duration"]}h'] = is_wet.rolling(window=mold_cfg["rh_duration"], min_periods=1).sum()
        df_out['dew_point'] = df_out['temperature'] - ((100 - df_out['humidity']) / 5)

        # IMPORTANT: Fill NaNs created during this process
        df_out.fillna(method='ffill', inplace=True)
        df_out.fillna(method='bfill', inplace=True)
        df_out.fillna(0, inplace=True)

        return df_out

    def _engineer_features(self):
        """Wrapper for feature engineering on the main dataframe."""
        print("--> STAGE 2: Advanced Feature Engineering...")
        self.df = self._generate_base_features(self.df)

        print("... Detecting anomalies with Isolation Forest...")
        original_numeric_cols = [
            'temperature', 'humidity', 'co2', 'pm2.5', 'pm10',
            'tvoc', 'co', 'light_intensity', 'occupancy_count'
        ]
        cols_to_use = [col for col in original_numeric_cols if col in self.df.columns]
        
        if_model = IsolationForest(contamination=self.config.get("processing", "anomaly_contamination"), random_state=SEED)
        self.df['anomaly_score'] = if_model.fit_predict(self.df[cols_to_use])

        print(f"Feature engineering complete. Total features: {len(self.df.columns)}")

    def _define_holistic_environmental_state(self):
        """Defines a comprehensive target variable based on scientific thresholds."""
        print("--> STAGE 3: Defining Holistic Environmental State (Target Variable)...")
        
        mold_cfg = self.config.get("microbial_thresholds", "mold")
        virus_cfg = self.config.get("microbial_thresholds", "virus_persistence")

        def get_state(row):
            issues = []
            if row['temperature'] > 28: issues.append('Temp-Hot')
            elif row['temperature'] < 20: issues.append('Temp-Cold')
            if row['humidity'] > 70: issues.append('Humidity-High')
            elif row['humidity'] < 30: issues.append('Humidity-Low')
            if 'co2' in row and row['co2'] > 1500: issues.append('Ventilation-Poor')
            elif 'co2' in row and row['co2'] > 1000: issues.append('Ventilation-Moderate')
            if 'pm2.5' in row and row['pm2.5'] > 35.5: issues.append('PM2.5-Unhealthy')
            elif 'pm2.5' in row and row['pm2.5'] > 12: issues.append('PM2.5-Moderate')

            is_temp_conducive = mold_cfg["temperature_range"][0] <= row['temperature'] <= mold_cfg["temperature_range"][1]
            if row[f'time_of_wetness_{mold_cfg["rh_duration"]}h'] >= mold_cfg["rh_duration"] and is_temp_conducive:
                issues.append('Risk-Mold')

            is_rh_risky = not (virus_cfg["rh_low_risk"] < row['humidity'] < virus_cfg["rh_high_risk"])
            is_vent_poor = 'co2' in row and row['co2'] > virus_cfg["co2_ventilation_proxy"]
            if is_rh_risky and is_vent_poor:
                issues.append('Risk-VirusPersistence')

            if not issues: return 'Optimal'
            return '_'.join(sorted(issues))

        self.df['holistic_state'] = self.df.apply(get_state, axis=1)
        print("Holistic environmental states defined.")
        state_distribution = self.df['holistic_state'].value_counts(normalize=True)
        self.report_data['state_distribution'] = state_distribution
        print(f"\nState Distribution (Top 15):\n{state_distribution.head(15).to_string()}")

    def _prepare_for_modeling(self):
        """Prepares data for modeling, including scaling, encoding, and resampling."""
        print("--> STAGE 4: Preparing Data for Modeling...")
        
        self.y = self.df['holistic_state']
        self.X = self.df.drop(columns=['holistic_state'])

        categorical_cols = self.X.select_dtypes(include=['object', 'category']).columns
        if not categorical_cols.empty:
            print(f"Found categorical columns to encode: {categorical_cols.tolist()}")
            self.X = pd.get_dummies(self.X, columns=categorical_cols, drop_first=True)
        
        self.feature_names = self.X.columns.tolist()

        self.encoder = LabelEncoder()
        y_encoded = self.encoder.fit_transform(self.y)

        train_ratio = self.config.get("model", "train_test_split")
        split_idx = int(len(self.X) * train_ratio)
        X_train, self.X_test = self.X.iloc[:split_idx], self.X.iloc[split_idx:]
        y_train, self.y_test = y_encoded[:split_idx], y_encoded[split_idx:]

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        self.X_test = self.scaler.transform(self.X_test)

        unique_classes, counts = np.unique(y_train, return_counts=True)
        k_neighbors = max(1, min(5, np.min(counts) - 1))
        
        if k_neighbors > 0:
            print(f"Handling class imbalance with SMOTE (k_neighbors={k_neighbors})...")
            smote = SMOTE(random_state=SEED, k_neighbors=k_neighbors)
            self.X_train, self.y_train = smote.fit_resample(X_train_scaled, y_train)
            print(f"Training data balanced. New shape: {self.X_train.shape}")
        else:
            print("WARNING: Cannot apply SMOTE, at least one class has fewer than 2 samples. Using original data.")
            self.X_train = X_train_scaled
            self.y_train = y_train

    def _train_and_save_model(self):
        """Tunes, trains, and saves an advanced stacking ensemble model."""
        print("--> STAGE 5: Hyperparameter Tuning & Advanced Model Training...")

        # Define base models
        xgb = XGBClassifier(
            objective='multi:softmax', num_class=len(self.encoder.classes_),
            random_state=SEED, n_estimators=200, learning_rate=0.1, max_depth=7,
            use_label_encoder=False, eval_metric='mlogloss'
        )
        rf = RandomForestClassifier(random_state=SEED, n_jobs=-1)

        # --- HYPERPARAMETER TUNING (DEMONSTRATION) ---
        # In a real project, this grid would be much larger.
        param_grid = {
            'n_estimators': [100, 150],
            'max_depth': [10, 20]
        }
        print(f"Tuning RandomForest with GridSearchCV... (Parameter grid: {param_grid})")
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1_weighted')
        grid_search.fit(self.X_train, self.y_train)
        best_rf = grid_search.best_estimator_
        print(f"Best RandomForest parameters found: {grid_search.best_params_}")
        
        # --- ADVANCED ENSEMBLE: STACKING ---
        estimators = [
            ('xgboost', xgb),
            ('random_forest', best_rf)
        ]
        # The meta-model learns to combine the predictions of the base models
        meta_model = LogisticRegression(solver='liblinear')
        
        stacking_classifier = StackingClassifier(
            estimators=estimators, final_estimator=meta_model, cv=5
        )
        
        print("Training the StackingClassifier...")
        stacking_classifier.fit(self.X_train, self.y_train)
        self.models['ensemble'] = stacking_classifier
        print("Stacking model trained successfully.")

        # Save the complete model bundle
        model_bundle = {
            'model': stacking_classifier,
            'scaler': self.scaler,
            'encoder': self.encoder,
            'feature_names': self.feature_names
        }
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        joblib.dump(model_bundle, model_path)
        print(f"Model bundle saved to: {model_path}")

    def _evaluate_model(self):
        """Evaluates the final model and generates and saves comprehensive evaluation plots."""
        print("--> STAGE 6: Model Evaluation and Visualization...")
        
        if 'ensemble' not in self.models:
            print("ERROR: No ensemble model found to evaluate.")
            return

        model = self.models['ensemble']
        y_pred = model.predict(self.X_test)
        y_prob = model.predict_proba(self.X_test)
        y_true_labels = self.encoder.inverse_transform(self.y_test)
        y_pred_labels = self.encoder.inverse_transform(y_pred)
        class_labels = self.encoder.classes_
        n_classes = len(class_labels)

        # 1. Save Classification Report to file and store for final report
        print("\n--- Classification Report ---\n")
        report_str = classification_report(y_true_labels, y_pred_labels, target_names=class_labels, zero_division=0)
        print(report_str)
        with open(os.path.join(self.output_dir, "classification_report.txt"), "w") as f:
            f.write(report_str)
        print("Classification report saved to classification_report.txt")
        self.report_data['classification_report'] = report_str
        report_dict = classification_report(y_true_labels, y_pred_labels, target_names=class_labels, zero_division=0, output_dict=True)

        # 2. Confusion Matrix
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(y_true_labels, y_pred_labels, labels=class_labels)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.title("Ensemble Model Confusion Matrix", fontsize=16)
        plt.xlabel("Predicted State", fontsize=12); plt.ylabel("Actual State", fontsize=12)
        plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "confusion_matrix.png"), dpi=300)
        plt.show()

        # 3. Model-based Feature Importance Plot
        print("Generating and saving model feature importances...")
        # Note: StackingClassifier doesn't have a single `feature_importances_` attribute.
        # We'll show the importances from the base XGBoost model as a proxy.
        xgb_model = model.named_estimators_['xgboost']
        xgb_imp = xgb_model.feature_importances_
        imp_df = pd.DataFrame({'feature': self.feature_names, 'importance': xgb_imp})
        imp_df = imp_df.sort_values('importance', ascending=False).head(20)
        self.report_data['feature_importances'] = imp_df
        
        plt.figure(figsize=(12, 8))
        imp_df.sort_values('importance').plot(x='feature', y='importance', kind='barh', figsize=(12, 10), legend=False)
        plt.title('Top 20 Feature Importances (from XGBoost component)')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "feature_importances.png"), dpi=300)
        plt.show()

        # 4. SHAP Summary Plots
        print("Generating and saving SHAP explanations...")
        explainer = shap.TreeExplainer(xgb_model)
        X_test_df = pd.DataFrame(self.X_test, columns=self.feature_names)
        shap_values = explainer(X_test_df)

        plt.figure(figsize=(10, 8))
        shap.plots.bar(shap_values, max_display=20, show=False)
        plt.title("SHAP Feature Importance (Mean Absolute SHAP Value)", fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "shap_bar_plot.png"), dpi=300)
        plt.show()

        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_test_df, max_display=20, show=False)
        plt.title("SHAP Feature Impact Summary", fontsize=16)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "shap_beeswarm_plot.png"), dpi=300)
        plt.show()

        # Binarize the output for PR and ROC curves
        y_test_binarized = label_binarize(self.y_test, classes=range(n_classes))

        # 5. Precision-Recall and ROC Curves
        if n_classes <= 15:
            plt.figure(figsize=(12, 8))
            for i in range(n_classes):
                precision, recall, _ = precision_recall_curve(y_test_binarized[:, i], y_prob[:, i])
                plt.plot(recall, precision, lw=2, label=f'PR curve for {class_labels[i]}')
            plt.xlabel("Recall"); plt.ylabel("Precision")
            plt.title("Precision-Recall Curve for Each Class"); plt.legend(loc="best")
            plt.grid(True); plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, "precision_recall_curves.png"), dpi=300)
            plt.show()

            plt.figure(figsize=(12, 8))
            for i in range(n_classes):
                fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
                roc_auc = auc(fpr, tpr)
                plt.plot(fpr, tpr, lw=2, label=f'ROC curve for {class_labels[i]} (area = {roc_auc:0.2f})')
            plt.plot([0, 1], [0, 1], 'k--', lw=2)
            plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
            plt.title("ROC Curve for Each Class"); plt.legend(loc="lower right")
            plt.grid(True); plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, "roc_curves.png"), dpi=300)
            plt.show()
        else:
            print("Skipping PR and ROC curves due to high number of classes, which would make the plot unreadable.")

        # 6. Overall Metrics Summary Plot
        metrics = {'Accuracy': report_dict['accuracy'], 'Macro Avg F1': report_dict['macro avg']['f1-score'], 'Weighted Avg F1': report_dict['weighted avg']['f1-score']}
        plt.figure(figsize=(8, 6))
        sns.barplot(x=list(metrics.keys()), y=list(metrics.values()))
        plt.title('Overall Model Performance Metrics'); plt.ylabel('Score'); plt.ylim(0, 1)
        for index, value in enumerate(metrics.values()):
            plt.text(index, value + 0.01, f"{value:.3f}", ha='center')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "metrics_summary.png"), dpi=300)
        plt.show()

    def _generate_research_paper_report(self):
        """Generates a detailed text report suitable for a research paper."""
        print("--> STAGE 7: Generating Research Paper Report...")
        
        report_path = os.path.join(self.output_dir, "research_report.txt")
        
        with open(report_path, "w") as f:
            f.write("="*80 + "\n")
            f.write("AIMRAS: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM - RESULTS\n")
            f.write("="*80 + "\n\n")

            # Section 1: Data Summary
            f.write("1. DATA SUMMARY\n")
            f.write("-"*80 + "\n")
            f.write(f"The analysis was conducted on the 'IoT Indoor Air Quality Dataset'.\n")
            f.write(f"Total observations after cleaning: {self.df.shape[0]}\n")
            f.write(f"Total features after engineering: {self.df.shape[1] - 1}\n\n")
            f.write("Distribution of Environmental States (Target Variable):\n")
            f.write(self.report_data['state_distribution'].to_string())
            f.write("\n\n")

            # Section 2: Model Performance
            f.write("2. MODEL PERFORMANCE\n")
            f.write("-"*80 + "\n")
            f.write("An advanced StackingClassifier ensemble model was trained. This model uses a Logistic Regression\n")
            f.write("meta-model to intelligently combine predictions from two base models: XGBoost and a tuned RandomForest.\n")
            f.write("Hyperparameter tuning (GridSearchCV) was performed on the RandomForest component.\n\n")
            f.write("The following metrics were achieved on the unseen test set:\n\n")
            f.write(self.report_data['classification_report'])
            f.write("\n\n")

            # Section 3: Feature Importance
            f.write("3. FEATURE IMPORTANCE ANALYSIS\n")
            f.write("-"*80 + "\n")
            f.write("The top 10 most influential features from the XGBoost component of the stack were:\n")
            top_10_features = self.report_data['feature_importances'].head(10)
            f.write(top_10_features[['feature', 'importance']].to_string(index=False))
            f.write("\n\n")
            
            # Conclusion
            f.write("4. CONCLUSION\n")
            f.write("-"*80 + "\n")
            f.write("The advanced stacking ensemble model demonstrated high performance in classifying holistic environmental states.\n")
            f.write("Key drivers for prediction included both raw sensor values (e.g., pm2.5) and engineered temporal features (e.g., hour_sin).\n")
            f.write("This indicates that both immediate conditions and time-based patterns are crucial for accurate assessment.\n")
            f.write("All supporting graphs (Confusion Matrix, Feature Importance, SHAP plots, etc.) are saved in the output directory.\n")

        print(f"✅ Research report saved to: {report_path}")

    def predict_on_new_data(self, new_data: pd.DataFrame):
        """Predicts the environmental state for new, unseen data using the saved model bundle."""
        print("--> PREDICTION: Running inference on new data...")
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        
        try:
            bundle = joblib.load(model_path)
            model, scaler, encoder, feature_names = bundle['model'], bundle['scaler'], bundle['encoder'], bundle['feature_names']
        except FileNotFoundError:
            print(f"ERROR: Model bundle not found at {model_path}. Please run the training pipeline first.")
            return None
        
        # Ensure index is datetime for feature generation
        if 'timestamp' in new_data.columns:
            new_data['timestamp'] = pd.to_datetime(new_data['timestamp'])
            new_data = new_data.set_index('timestamp')
        else:
            new_data.index = [pd.to_datetime(pd.Timestamp.now())]

        # Generate the same features as the training pipeline
        new_data_featured = self._generate_base_features(new_data)
        
        # Add anomaly_score feature (assuming new data is not an anomaly)
        new_data_featured['anomaly_score'] = 1
        
        # Handle categorical columns in the new data
        categorical_cols = new_data_featured.select_dtypes(include=['object', 'category']).columns
        if not categorical_cols.empty:
            new_data_featured = pd.get_dummies(new_data_featured, columns=categorical_cols, drop_first=True)

        new_data_processed = new_data_featured.reindex(columns=feature_names, fill_value=0)
        new_data_scaled = scaler.transform(new_data_processed)
        
        predictions_encoded = model.predict(new_data_scaled)
        predictions_proba = model.predict_proba(new_data_scaled)
        predictions_labels = encoder.inverse_transform(predictions_encoded)
        
        results = []
        for i, label in enumerate(predictions_labels):
            confidence = predictions_proba[i][predictions_encoded[i]]
            results.append({"predicted_state": label, "confidence": f"{confidence:.2%}"})
            print(f"Prediction for sample {i}: {label} (Confidence: {confidence:.2%})")
            
        return results

    def run_full_pipeline(self):
        """Executes the entire analytics pipeline from start to finish."""
        print("🚀 STARTING AIMRAS FULL ANALYTICS PIPELINE 🚀")
        try:
            # --- Main Pipeline ---
            stages = [
                ("Loading and Cleaning Data", self._load_and_clean_data, self.config.get("data", "historical_file_path")),
                ("Engineering Features", self._engineer_features, None),
                ("Defining Holistic Environmental State", self._define_holistic_environmental_state, None),
                ("Preparing Data for Modeling", self._prepare_for_modeling, None),
                ("Training and Saving Model", self._train_and_save_model, None),
                ("Evaluating Model", self._evaluate_model, None),
                ("Generating Research Paper Report", self._generate_research_paper_report, None)
            ]

            for i, (name, func, arg) in enumerate(stages):
                print(f"\n[STAGE {i+1}/{len(stages)}] {name}...")
                if arg:
                    func(arg)
                else:
                    func()
                print(f"✅ Stage {i+1} Complete")

            print("\n\n🎉 AIMRAS FULL ANALYTICS PIPELINE COMPLETED SUCCESSFULLY! 🎉")
            print(f"All outputs saved in the '{self.output_dir}' directory.")

        except Exception:
            print(f"\n❌ PIPELINE FAILED! See the error traceback below.")
            traceback.print_exc()

# ------------------------------------------------------------------------------
# SECTION D: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    engine = EnvironmentalDiagnosticEngine()
    engine.run_full_pipeline()

    if os.path.exists(os.path.join(engine.output_dir, 'aimras_model_bundle.pkl')):
        print("\n" + "="*50)
        print("DEMONSTRATING PREDICTION ON NEW (TEST) DATA")
        print("="*50)
        
        # To simulate new data, we create a dummy DataFrame with only raw inputs.
        sample_data = {
            'timestamp': [pd.Timestamp.now()],
            'temperature': [25.5], 
            'humidity': [85.0], 
            'co2': [1600], 
            'pm2.5': [40.0],
            'motion_detected': [True], 
            'ventilation_status': ['Open'] 
        }
        sample_df = pd.DataFrame(sample_data)
        
        # The prediction function will handle all feature engineering and encoding
        predictions = engine.predict_on_new_data(sample_df)


In [ ]:
import shutil
from IPython.display import FileLink

# --- CONFIGURATION ---
OUTPUT_DIR = "/kaggle/working/"
OUTPUT_ZIP = "AIMRAS V13.zip"

# --- ZIP THE DIRECTORY ---
shutil.make_archive(base_name=OUTPUT_ZIP.replace(".zip", ""), format="zip", root_dir=OUTPUT_DIR)

# --- DISPLAY DOWNLOAD LINK ---
print(f"✅ Zipped {OUTPUT_DIR} to {OUTPUT_ZIP}")
display(FileLink(OUTPUT_ZIP))

In [ ]:
!pip install --quiet \
     numpy==1.24.3 \
     pandas==2.0.3 \
     scikit-learn==1.3.2 \
     scipy==1.11.4 \
     xgboost==2.0.3 \
     shap==0.45.1 \
     seaborn==0.13.2 \
     matplotlib==3.8.4 \
     joblib==1.4.2

In [ ]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (AIMRAS)
# VERSION: 11.2 (Robust Training Engine)
#
# KEY ENHANCEMENTS FROM PREVIOUS VERSION:
# 1. XGBOOST ERROR FIX:
#    - Resolved the `xgboost.core.XGBoostError` by removing the invalid parameter
#      `scale_pos_weight='auto'`. The model now uses its default robust
#      handling for class weights.
#
# 2. AUTOMATIC LABEL PRUNING (CRITICAL FIX):
#    - Implemented a pre-training check to identify and exclude target labels
#      that have no class variation (i.e., all 0s or all 1s).
#    - This prevents the model from crashing when encountering data where certain
#      risks never occur, making the pipeline significantly more robust.
#
# 3. ENHANCED LOGGING:
#    - Added clear print statements to inform the user which labels were
#      pruned due to a lack of data, improving transparency.
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# Core package installation with a known-stable set of pinned versions.
# !pip install --quiet \
#      numpy==1.24.3 \
#      pandas==2.0.3 \
#      scikit-learn==1.3.2 \
#      scipy==1.11.4 \
#      xgboost==2.0.3 \
#      shap==0.45.1 \
#      seaborn==0.13.2 \
#      matplotlib==3.8.4 \
#      joblib==1.4.2

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import warnings
import json
import os
import re
import traceback
from typing import Dict, List, Tuple, Any

# Scikit-learn, XGBoost
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, jaccard_score, hamming_loss
)
from sklearn.multioutput import ClassifierChain
from xgboost import XGBClassifier

# Silence non-critical warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)

# ------------------------------------------------------------------------------
# SECTION B: CONFIGURATION SYSTEM
# ------------------------------------------------------------------------------

class AIMRASConfig:
    """Manages configuration for the AIMRAS analytics script."""
    DEFAULT_CONFIG = {
        "data": {
            "historical_file_path": "/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv",
        },
        "processing": {
             "outlier_capping_factor": 2.5 # IQR multiplier for capping outliers
        },
        "model": {
            "train_test_split": 0.8,
            "output_dir": "aimras_outputs_v11" # Directory for saved models and graphs
        },
        "risk_thresholds": {
            "mold_growth": {
                "rh_sustained": 75.0,  # % RH - Lowered for more sensitivity
                "rh_duration": 48,      # hours
                "temperature_range": [5, 35],  # °C
            },
            "virus_persistence": {
                "rh_optimal_range": [40, 60], # % RH range where persistence is lowest
                "co2_ventilation_proxy": 1200 # ppm, high CO2 indicates poor ventilation
            },
            "particulate_matter": {
                "pm2.5_high": 35.4, # Unhealthy for sensitive groups
                "pm2.5_moderate": 12.0
            },
            "ventilation": {
                "co2_poor": 1500, # ppm
                "co2_moderate": 1000 # ppm
            },
            "thermal_comfort": {
                "temp_hot": 27.0, # °C
                "temp_cold": 20.0 # °C
            },
            "humidity_comfort": {
                "rh_high": 65.0, # %
                "rh_low": 30.0 # %
            }
        }
    }

    def __init__(self, config_file=None):
        self.config = self.DEFAULT_CONFIG.copy()
        if config_file and os.path.exists(config_file):
            with open(config_file, 'r') as f:
                file_config = json.load(f)
                self._deep_update(self.config, file_config)
            print(f"Loaded configuration from {config_file}")
        print("Configuration initialized.")

    def _deep_update(self, d, u):
        for k, v in u.items():
            if isinstance(v, dict) and k in d and isinstance(d[k], dict):
                self._deep_update(d[k], v)
            else:
                d[k] = v

    def get(self, section, key=None):
        if key is None:
            return self.config.get(section, {})
        return self.config.get(section, {}).get(key)

# ------------------------------------------------------------------------------
# SECTION C: CORE ANALYTICS ENGINE
# ------------------------------------------------------------------------------

class EnvironmentalDiagnosticEngine:
    """
    A comprehensive engine for indoor air quality analysis, featuring a multi-label
    diagnostic approach to identify specific environmental risks.
    """
    def __init__(self, config_file=None):
        self.config = AIMRASConfig(config_file)
        self.df = None
        self.X_train, self.X_test = None, None
        self.y_train, self.y_test = None, None
        self.model = None
        self.scaler = None
        self.feature_names = None
        self.target_labels = None
        self.report_data = {}
        self.output_dir = self.config.get("model", "output_dir")
        os.makedirs(self.output_dir, exist_ok=True)
        print(f"✅ EnvironmentalDiagnosticEngine initialized. Outputs will be saved to '{self.output_dir}/'")

    def _load_and_clean_data(self, filepath: str):
        """Loads and cleans the dataset with robust validation and processing."""
        print("--> STAGE 1: Loading and Cleaning Data...")

        if not os.path.exists(filepath):
            error_message = (
                f"FATAL ERROR: Dataset not found at {filepath}\n\n"
                "--- How to fix on Kaggle: ---\n"
                "1. Click '+ Add data' in the right-hand pane.\n"
                "2. Search for 'IoT Indoor Air Quality Dataset' and add it."
            )
            raise FileNotFoundError(error_message)

        try:
            self.df = pd.read_csv(filepath)
            print(f"Loaded {len(self.df)} rows from {filepath}")

            def clean_col_name(column_name):
                name = re.sub(r'\(.*\)', '', column_name)
                name = name.strip().lower().replace(' ', '_').replace('-', '_')
                return name

            self.df.rename(columns=clean_col_name, inplace=True)
            print(f"Standardized column names: {self.df.columns.tolist()}")

            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'], errors='coerce')
            self.df = self.df.dropna(subset=['timestamp']).set_index('timestamp').sort_index()
            self.df = self.df[~self.df.index.duplicated(keep='last')]

            numeric_cols = self.df.select_dtypes(include=np.number).columns
            self.df[numeric_cols] = self.df[numeric_cols].interpolate(method='time')
            self.df = self.df.ffill().bfill()

            capping_factor = self.config.get("processing", "outlier_capping_factor")
            for col in numeric_cols:
                Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower, upper = Q1 - capping_factor * IQR, Q3 + capping_factor * IQR
                if ((self.df[col] < lower) | (self.df[col] > upper)).any():
                    print(f"Capping outliers in '{col}'.")
                    self.df[col] = self.df[col].clip(lower, upper)

            print(f"Data cleaned successfully. Final shape: {self.df.shape}")

        except Exception as e:
            print(f"ERROR during data loading/cleaning: {e}")
            raise

    def _engineer_features(self):
        """Generates advanced features for time-series and interaction analysis."""
        print("--> STAGE 2: Advanced Feature Engineering...")
        df_out = self.df.copy()

        # Time-based features
        df_out['hour'] = df_out.index.hour
        df_out['day_of_week'] = df_out.index.dayofweek
        df_out['is_weekend'] = df_out['day_of_week'].isin([5, 6]).astype(int)
        df_out['hour_sin'] = np.sin(2 * np.pi * df_out['hour'] / 24)
        df_out['hour_cos'] = np.cos(2 * np.pi * df_out['hour'] / 24)

        # Rolling statistics & Lag features
        key_sensors = ['temperature', 'humidity', 'co2', 'pm2.5']
        for col in key_sensors:
            if col in df_out.columns:
                df_out[f'{col}_24h_mean'] = df_out[col].rolling('24H', min_periods=1).mean()
                df_out[f'{col}_1h_lag'] = df_out[col].shift(1)
                df_out[f'{col}_rate_of_change'] = df_out[col].diff()

        # Interaction & Physical Features
        df_out['temp_humidity_interaction'] = df_out['temperature'] * df_out['humidity']
        df_out['dew_point'] = df_out['temperature'] - ((100 - df_out['humidity']) / 5)
        df_out['dew_point_spread'] = df_out['temperature'] - df_out['dew_point']

        # Fill NaNs created during feature engineering
        df_out = df_out.ffill().bfill()
        df_out.fillna(0, inplace=True)

        self.df = df_out
        print(f"Feature engineering complete. Total features: {len(self.df.columns)}")

    def _define_risk_and_comfort_labels(self):
        """Defines a comprehensive suite of binary risk and comfort labels for multi-label classification."""
        print("--> STAGE 3: Defining Comprehensive Risk & Comfort States (Targets)...")
        
        thresholds = self.config.get("risk_thresholds")
        df = self.df

        # Risk 1: Mold Growth Potential
        mold_cfg = thresholds['mold_growth']
        is_wet = (df['humidity'] > mold_cfg["rh_sustained"]).astype(int)
        time_of_wetness = is_wet.rolling(window=f'{mold_cfg["rh_duration"]}H').sum()
        is_temp_conducive = df['temperature'].between(mold_cfg["temperature_range"][0], mold_cfg["temperature_range"][1])
        df['Risk_Mold_Growth'] = ((time_of_wetness >= mold_cfg["rh_duration"]) & is_temp_conducive).astype(int)

        # Risk 2: Airborne Virus Persistence
        virus_cfg = thresholds['virus_persistence']
        is_rh_risky = ~df['humidity'].between(virus_cfg["rh_optimal_range"][0], virus_cfg["rh_optimal_range"][1])
        is_vent_poor_for_virus = df['co2'] > virus_cfg["co2_ventilation_proxy"]
        df['Risk_Virus_Persistence'] = (is_rh_risky & is_vent_poor_for_virus).astype(int)

        # Risk 3 & 4: Particulate Matter (High and Moderate)
        pm_cfg = thresholds['particulate_matter']
        df['Risk_High_PM2.5'] = (df['pm2.5'] > pm_cfg['pm2.5_high']).astype(int)
        df['Risk_Moderate_PM2.5'] = (df['pm2.5'].between(pm_cfg['pm2.5_moderate'], pm_cfg['pm2.5_high'])).astype(int)

        # Risk 5 & 6: Ventilation (Poor and Moderate)
        vent_cfg = thresholds['ventilation']
        df['Risk_Poor_Ventilation'] = (df['co2'] > vent_cfg['co2_poor']).astype(int)
        df['Risk_Moderate_Ventilation'] = (df['co2'].between(vent_cfg['co2_moderate'], vent_cfg['co2_poor'])).astype(int)

        # Comfort 7 & 8: Thermal Comfort
        thermal_cfg = thresholds['thermal_comfort']
        df['Comfort_Thermal_Hot'] = (df['temperature'] > thermal_cfg['temp_hot']).astype(int)
        df['Comfort_Thermal_Cold'] = (df['temperature'] < thermal_cfg['temp_cold']).astype(int)

        # Comfort 9 & 10: Humidity Comfort
        humidity_cfg = thresholds['humidity_comfort']
        df['Comfort_Humidity_High'] = (df['humidity'] > humidity_cfg['rh_high']).astype(int)
        df['Comfort_Humidity_Low'] = (df['humidity'] < humidity_cfg['rh_low']).astype(int)

        all_labels = [
            'Risk_Mold_Growth', 'Risk_Virus_Persistence',
            'Risk_High_PM2.5', 'Risk_Moderate_PM2.5',
            'Risk_Poor_Ventilation', 'Risk_Moderate_Ventilation',
            'Comfort_Thermal_Hot', 'Comfort_Thermal_Cold',
            'Comfort_Humidity_High', 'Comfort_Humidity_Low'
        ]
        self.df.dropna(subset=all_labels, inplace=True)

        print("Comprehensive labels defined. Initial distribution of states:")
        distribution = self.df[all_labels].sum() / len(self.df)
        self.report_data['initial_risk_distribution'] = distribution
        print(distribution)

        # --- ROBUSTNESS FIX: Prune labels with no variation ---
        self.target_labels = []
        for label in all_labels:
            if self.df[label].nunique() > 1:
                self.target_labels.append(label)
            else:
                print(f"WARNING: Pruning label '{label}' due to no variation (all values are the same).")
        
        print(f"\nFinal set of {len(self.target_labels)} trainable labels: {self.target_labels}")


    def _prepare_for_modeling(self):
        """Prepares data for multi-label modeling."""
        print("--> STAGE 4: Preparing Data for Modeling...")

        # Drop original sensor columns that are now represented by lags/means
        # and categorical columns before defining X
        cols_to_drop_for_X = ['ventilation_status']
        self.df.drop(columns=[col for col in cols_to_drop_for_X if col in self.df.columns], inplace=True)

        self.y = self.df[self.target_labels]
        self.X = self.df.drop(columns=self.target_labels)
        
        # Align columns - crucial for prediction
        self.feature_names = self.X.columns.tolist()

        train_ratio = self.config.get("model", "train_test_split")
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, train_size=train_ratio, shuffle=False # Time-series split
        )

        self.scaler = StandardScaler()
        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
        print(f"Data prepared. Train shape: {self.X_train.shape}, Test shape: {self.X_test.shape}")

    def _train_and_save_model(self):
        """Trains a ClassifierChain model for multi-label prediction."""
        print("--> STAGE 5: Training and Saving Model...")

        # --- FIX: Removed `scale_pos_weight='auto'` as it's an invalid parameter ---
        # XGBoost's default behavior is generally robust. For highly imbalanced data,
        # other strategies like over/under-sampling or explicitly calculating
        # weights would be needed, but this fix resolves the crash.
        base_classifier = XGBClassifier(
            objective='binary:logistic',
            random_state=SEED,
            n_estimators=150,
            learning_rate=0.05,
            max_depth=5,
            use_label_encoder=False,
            eval_metric='logloss'
        )

        # Use ClassifierChain to model label dependencies
        self.model = ClassifierChain(base_classifier, order='random', random_state=SEED)

        print("Training the ClassifierChain with XGBoost base estimator...")
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully.")

        model_bundle = {
            'model': self.model,
            'scaler': self.scaler,
            'feature_names': self.feature_names,
            'target_labels': self.target_labels
        }
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        joblib.dump(model_bundle, model_path)
        print(f"Model bundle saved to: {model_path}")

    def _evaluate_model(self):
        """Evaluates the multi-label model and generates per-risk explanations."""
        print("--> STAGE 6: Model Evaluation and Visualization...")
        
        if not self.model:
            print("ERROR: No model found to evaluate.")
            return

        y_pred = self.model.predict(self.X_test)
        y_prob = self.model.predict_proba(self.X_test)

        # Convert predictions to DataFrame for easier handling
        y_pred_df = pd.DataFrame(y_pred, columns=self.target_labels, index=self.y_test.index)

        # 1. Overall Multi-Label Metrics
        print("\n--- Overall Multi-Label Metrics ---")
        j_score = jaccard_score(self.y_test, y_pred_df, average='samples')
        h_loss = hamming_loss(self.y_test, y_pred_df)
        print(f"Sample-Averaged Jaccard Score: {j_score:.4f}")
        print(f"Hamming Loss (fraction of wrong labels): {h_loss:.4f}")
        self.report_data['overall_metrics'] = {'jaccard_score': j_score, 'hamming_loss': h_loss}

        # 2. Per-Label Evaluation and SHAP Analysis
        X_test_df = pd.DataFrame(self.X_test, columns=self.feature_names)

        for i, label in enumerate(self.target_labels):
            print(f"\n--- Evaluating and Explaining: {label} ---")
            
            # A. Classification Report and Confusion Matrix
            report_str = classification_report(self.y_test[label], y_pred_df[label], zero_division=0)
            print(report_str)
            self.report_data[f'report_{label}'] = report_str

            plt.figure(figsize=(6, 5))
            cm = confusion_matrix(self.y_test[label], y_pred_df[label])
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
            plt.title(f"Confusion Matrix for {label}", fontsize=14)
            plt.xlabel("Predicted"); plt.ylabel("Actual")
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, f"confusion_matrix_{label}.png"), dpi=300)
            plt.show()

            # B. SHAP Analysis for this specific risk
            print(f"Generating SHAP explanations for {label}...")
            estimator = self.model.estimators_[i]
            explainer = shap.TreeExplainer(estimator)
            
            shap_values = explainer(X_test_df)

            # SHAP Bar Plot (Global Importance)
            plt.figure()
            shap.plots.bar(shap_values, max_display=15, show=False)
            plt.title(f"Feature Importance for {label}", fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, f"shap_bar_plot_{label}.png"), dpi=300)
            plt.show()

            # SHAP Beeswarm Plot (Detailed Impact)
            plt.figure()
            shap.summary_plot(shap_values, X_test_df, max_display=15, show=False)
            plt.title(f"SHAP Feature Impact on {label}", fontsize=14)
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, f"shap_beeswarm_plot_{label}.png"), dpi=300)
            plt.show()

    def _generate_diagnostic_report(self):
        """Generates a detailed text report suitable for diagnostics."""
        print("--> STAGE 7: Generating Diagnostic Report...")
        
        report_path = os.path.join(self.output_dir, "diagnostic_report.txt")
        
        with open(report_path, "w") as f:
            f.write("="*80 + "\n")
            f.write("AIMRAS: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT - DIAGNOSTIC REPORT\n")
            f.write("="*80 + "\n\n")

            f.write("1. EXECUTIVE SUMMARY\n")
            f.write("-"*80 + "\n")
            f.write("This report details the performance and findings of the AIMRAS multi-label diagnostic model.\n")
            f.write(f"The system was trained to identify {len(self.target_labels)} specific, co-occurring environmental risks and comfort states.\n")
            f.write(f"Overall Jaccard Score (prediction set similarity): {self.report_data['overall_metrics']['jaccard_score']:.4f}\n")
            f.write(f"Overall Hamming Loss (label prediction error rate): {self.report_data['overall_metrics']['hamming_loss']:.4f}\n\n")

            f.write("2. DATA & RISK PROFILE\n")
            f.write("-"*80 + "\n")
            f.write(f"Analysis conducted on {self.df.shape[0]} observations with {len(self.feature_names)} features.\n")
            f.write("Initial prevalence of each state in the dataset (including untrainable states):\n")
            f.write(self.report_data['initial_risk_distribution'].to_string())
            f.write("\n\n")

            for label in self.target_labels:
                f.write(f"3. DETAILED ANALYSIS: {label.upper()}\n")
                f.write("-"*80 + "\n")
                f.write("Performance on the test set:\n")
                f.write(self.report_data[f'report_{label}'])
                f.write("\n")

                # Find top SHAP features for this label
                estimator = self.model.estimators_[self.target_labels.index(label)]
                shap_values = shap.TreeExplainer(estimator).shap_values(pd.DataFrame(self.X_test, columns=self.feature_names))
                
                feature_importance = pd.DataFrame({
                    'feature': self.feature_names,
                    'mean_abs_shap': np.abs(shap_values).mean(axis=0)
                }).sort_values('mean_abs_shap', ascending=False)
                
                f.write("Top 5 Predictive Factors (from SHAP analysis):\n")
                f.write(feature_importance.head(5).to_string(index=False))
                f.write("\n\n")

            f.write("4. CONCLUSION\n")
            f.write("-"*80 + "\n")
            f.write("The ClassifierChain model effectively identifies a comprehensive set of environmental states.\n")
            f.write("Per-state SHAP analysis reveals distinct drivers for each condition, enabling targeted interventions.\n")
            f.write("All supporting graphs are saved in the output directory.\n")

        print(f"✅ Diagnostic report saved to: {report_path}")

    def predict_on_new_data(self, new_data: pd.DataFrame):
        """Predicts risk factors for new, unseen data using the saved model bundle."""
        print("--> PREDICTION: Running inference on new data...")
        model_path = os.path.join(self.output_dir, 'aimras_model_bundle.pkl')
        
        try:
            bundle = joblib.load(model_path)
            model, scaler, feature_names, target_labels = bundle['model'], bundle['scaler'], bundle['feature_names'], bundle['target_labels']
        except FileNotFoundError:
            print(f"ERROR: Model bundle not found at {model_path}. Please run the training pipeline first.")
            return None
        
        # Pre-process new data
        if 'timestamp' in new_data.columns:
            new_data['timestamp'] = pd.to_datetime(new_data['timestamp'])
            new_data = new_data.set_index('timestamp')
        else:
            # If no timestamp, create one for feature generation compatibility
            new_data.index = pd.to_datetime(pd.date_range(start=pd.Timestamp.now(), periods=len(new_data), freq='H'))

        # Clean column names to match training
        new_data.rename(columns=lambda c: re.sub(r'\(.*\)', '', c).strip().lower().replace(' ', '_').replace('-', '_'), inplace=True)
        
        # Create a dummy history for rolling features if not available
        # In a real application, you'd feed historical data
        if len(new_data) == 1:
            history_len = 24
            temp_history = pd.concat([new_data] * history_len, ignore_index=True)
            temp_history.index = pd.date_range(end=new_data.index[0], periods=history_len, freq='H')
            new_data_featured = self._engineer_features_for_prediction(temp_history).iloc[-1:]
        else:
            new_data_featured = self._engineer_features_for_prediction(new_data)
        
        new_data_processed = new_data_featured.reindex(columns=feature_names, fill_value=0)
        new_data_scaled = scaler.transform(new_data_processed)
        
        predictions_array = model.predict(new_data_scaled)
        
        results = []
        for i, idx in enumerate(new_data.index):
            result = {"timestamp": idx.isoformat()}
            for j, label in enumerate(target_labels):
                result[label] = "Yes" if predictions_array[i, j] == 1 else "No"
            results.append(result)
            print(f"Prediction for {idx}: {result}")
            
        return results

    def _engineer_features_for_prediction(self, df: pd.DataFrame) -> pd.DataFrame:
        """Feature engineering pipeline specifically for prediction to avoid data leakage."""
        df_out = df.copy()
        df_out['hour'] = df_out.index.hour
        df_out['day_of_week'] = df_out.index.dayofweek
        df_out['is_weekend'] = df_out['day_of_week'].isin([5, 6]).astype(int)
        df_out['hour_sin'] = np.sin(2 * np.pi * df_out['hour'] / 24)
        df_out['hour_cos'] = np.cos(2 * np.pi * df_out['hour'] / 24)
        key_sensors = ['temperature', 'humidity', 'co2', 'pm2.5']
        for col in key_sensors:
            if col in df_out.columns:
                df_out[f'{col}_24h_mean'] = df_out[col].rolling('24H', min_periods=1).mean()
                df_out[f'{col}_1h_lag'] = df_out[col].shift(1)
                df_out[f'{col}_rate_of_change'] = df_out[col].diff()
        df_out['temp_humidity_interaction'] = df_out.get('temperature', 0) * df_out.get('humidity', 0)
        df_out['dew_point'] = df_out.get('temperature', 0) - ((100 - df_out.get('humidity', 0)) / 5)
        df_out['dew_point_spread'] = df_out.get('temperature', 0) - df_out['dew_point']
        df_out = df_out.ffill().bfill().fillna(0)
        return df_out

    def run_full_pipeline(self):
        """Executes the entire analytics pipeline from start to finish."""
        print("🚀 STARTING AIMRAS V11 FULL ANALYTICS PIPELINE 🚀")
        try:
            stages = [
                ("Loading and Cleaning Data", self._load_and_clean_data, self.config.get("data", "historical_file_path")),
                ("Engineering Features", self._engineer_features, None),
                ("Defining Risk Labels", self._define_risk_and_comfort_labels, None),
                ("Preparing Data for Modeling", self._prepare_for_modeling, None),
                ("Training and Saving Model", self._train_and_save_model, None),
                ("Evaluating Model", self._evaluate_model, None),
                ("Generating Diagnostic Report", self._generate_diagnostic_report, None)
            ]

            for i, (name, func, arg) in enumerate(stages):
                print(f"\n[STAGE {i+1}/{len(stages)}] {name}...")
                if arg: func(arg)
                else: func()
                print(f"✅ Stage {i+1} Complete")

            print(f"\n\n🎉 AIMRAS V11 PIPELINE COMPLETED SUCCESSFULLY! 🎉")
            print(f"All outputs saved in the '{self.output_dir}' directory.")

        except Exception:
            print(f"\n❌ PIPELINE FAILED! See the error traceback below.")
            traceback.print_exc()

# ------------------------------------------------------------------------------
# SECTION D: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    engine = EnvironmentalDiagnosticEngine()
    engine.run_full_pipeline()

    # Demonstrate prediction on new data if the model was trained successfully
    model_path = os.path.join(engine.output_dir, 'aimras_model_bundle.pkl')
    if os.path.exists(model_path):
        print("\n" + "="*50)
        print("DEMONSTRATING PREDICTION ON NEW (SIMULATED) DATA")
        print("="*50)
        
        # Simulate a scenario conducive to multiple risks
        sample_data = {
            'timestamp': [pd.Timestamp.now()],
            'temperature': [25.5],
            'humidity': [85.0], # High humidity
            'co2': [1600],       # Poor ventilation
            'pm2.5': [40.0],     # High PM2.5
            'pm10': [55.0],
            'tvoc': [300],
            'co': [8.0],
            'light_intensity': [200],
            'motion_detected': [True],
            'occupancy_count': [3]
        }
        sample_df = pd.DataFrame(sample_data)
        
        predictions = engine.predict_on_new_data(sample_df)


In [ ]:
# Safe upgrade of required packages for AIMRAS
!pip install --quiet --upgrade \
    numpy \
    pandas \
    scikit-learn \
    tensorflow \
    joblib \
    shap \
    seaborn \
    plotly \
    pywavelets \
    matplotlib \
    imbalanced-learn \
    xgboost || echo "⚠️ Dependency conflicts detected with unused packages (e.g., ydata-profiling, bigframes). Safe to ignore."

# Restart kernel to apply package changes
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 111.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 104.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.7 MB/s eta 0:00:000:00:

In [ ]:
# ==============================================================================
# PROJECT: ADVANCED INDOOR MICROBIAL RISK ASSESSMENT SYSTEM (AIMRAS)
# FILE: aimras_lstm_trainer.py
# VERSION: 12.8 (Regularized Deep Training Edition)
#
# PURPOSE:
# This script trains a deeper, more heavily regularized Conv-LSTM network to
# combat the overfitting observed in the previous version.
#
# KEY ARCHITECTURE CHANGES & FIXES:
# 1. DEEPER ARCHITECTURE: The network has been made deeper with additional
#    Conv1D and Bidirectional LSTM layers to learn more complex and abstract
#    feature hierarchies, improving its ability to generalize.
# 2. ADVANCED REGULARIZATION (CRITICAL FIX):
#    - L2 Kernel Regularization: Added `l2(1e-4)` regularization to all key
#      layers to penalize large weights and prevent the model from relying on
#      any single feature.
#    - Increased Dropout: Dropout rates have been increased throughout the
#      network to further force the model to learn robust, redundant features.
# 3. ANTI-OVERFITTING FOCUS: These changes directly target the overfitting
#    problem, where the model was memorizing the synthetic training data but
#    failing to perform well on unseen validation data.
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION A: SETUP AND ENVIRONMENT
# ------------------------------------------------------------------------------
# Core package installation. imbalanced-learn is now required.
# !pip install --quiet \
#      numpy==1.24.3 \
#      pandas==2.0.3 \
#      scikit-learn==1.3.2 \
#      tensorflow==2.12.0 \
#      imbalanced-learn==0.11.0 \
#      joblib==1.4.2 \
#      matplotlib==3.8.4

# Core imports
import numpy as np
import pandas as pd
import joblib
import os
import re
import warnings
from typing import List, Tuple
import matplotlib.pyplot as plt

# Scikit-learn, Imblearn, and TensorFlow/Keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Bidirectional, BatchNormalization, 
    Conv1D, MaxPooling1D
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, Precision, Recall

# Silence non-critical warnings and set seeds for reproducibility
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ------------------------------------------------------------------------------
# SECTION B: DATA PROCESSING AND FEATURE ENGINEERING
# ------------------------------------------------------------------------------

def load_and_clean_data(filepath: str) -> pd.DataFrame:
    """Loads, cleans, and standardizes the dataset."""
    print("--> Loading and Cleaning Data...")
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Dataset not found at {filepath}")

    df = pd.read_csv(filepath)
    
    def clean_col_name(col_name):
        name = re.sub(r'\(.*\)', '', col_name).strip().lower().replace(' ', '_')
        return name
    
    df.rename(columns=clean_col_name, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp']).set_index('timestamp').sort_index()
    df = df[~df.index.duplicated(keep='last')]
    
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    print(f"Utilizing all {len(numeric_cols)} numeric columns: {numeric_cols}")
    
    df[numeric_cols] = df[numeric_cols].interpolate(method='time').ffill().bfill()
    return df

def define_risk_labels(df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str]]:
    """Defines a comprehensive suite of binary risk labels."""
    print("--> Defining Risk Labels...")
    df['Risk_High_PM2.5'] = (df['pm2.5'] > 35.4).astype(int)
    df['Risk_High_CO'] = (df['co'] > 9.0).astype(int)
    df['Risk_High_TVOC'] = (df['tvoc'] > 500).astype(int)
    df['Risk_Poor_Ventilation'] = (df['co2'] > 1500).astype(int)
    df['Comfort_Humidity_High'] = (df['humidity'] > 65.0).astype(int)
    df['Comfort_Humidity_Low'] = (df['humidity'] < 30.0).astype(int)
    df['Comfort_Thermal_Hot'] = (df['temperature'] > 27.0).astype(int)
    df['Comfort_Thermal_Cold'] = (df['temperature'] < 20.0).astype(int)

    all_labels = [
        'Risk_High_PM2.5', 'Risk_High_CO', 'Risk_High_TVOC',
        'Risk_Poor_Ventilation', 'Comfort_Humidity_High', 'Comfort_Humidity_Low',
        'Comfort_Thermal_Hot', 'Comfort_Thermal_Cold'
    ]
    
    trainable_labels = [label for label in all_labels if df[label].nunique() > 1]
    print(f"Found {len(trainable_labels)} trainable labels: {trainable_labels}")
    
    return df, trainable_labels

def create_sequences(X: np.ndarray, y: np.ndarray, time_steps: int) -> Tuple[np.ndarray, np.ndarray]:
    """Converts data into sequences for LSTM training."""
    print(f"--> Creating sequences with {time_steps} time steps...")
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# ------------------------------------------------------------------------------
# SECTION C: ADVANCED MODEL DEFINITION AND TRAINING
# ------------------------------------------------------------------------------

def build_conv_lstm_model(input_shape: tuple, num_labels: int) -> tf.keras.Model:
    """Builds the deeper, regularized Conv-LSTM model."""
    print("--> Building Deeper, Regularized Conv-LSTM model...")
    
    l2_reg = 1e-4 # Define the regularization factor
    
    inputs = Input(shape=input_shape)
    
    # --- Convolutional Feature Extraction Block ---
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(inputs)
    x = BatchNormalization()(x)
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x) # Increased dropout

    x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x) # Increased dropout

    # --- LSTM Sequence Processing Block ---
    x = Bidirectional(LSTM(units=128, return_sequences=True, kernel_regularizer=l2(l2_reg)))(x)
    x = Dropout(0.5)(x)
    
    x = Bidirectional(LSTM(units=64, kernel_regularizer=l2(l2_reg)))(x)
    x = Dropout(0.5)(x)
    
    # --- Final Prediction Block ---
    x = Dense(units=64, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(units=num_labels, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), # Slightly lower learning rate
        loss='binary_crossentropy',
        metrics=[
            AUC(name='auc'),
            Precision(name='precision'),
            Recall(name='recall')
        ]
    )
    model.summary()
    return model

def plot_and_save_history(history: tf.keras.callbacks.History, output_dir: str):
    """Plots training history and saves the figures."""
    print("--> Generating and saving training history graphs...")
    metrics = ['loss', 'auc', 'precision', 'recall']
    plt.style.use('seaborn-v0_8-whitegrid')

    for metric in metrics:
        plt.figure(figsize=(10, 6))
        plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.title(f'Training and Validation {metric.capitalize()}', fontsize=16)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel(metric.capitalize(), fontsize=12)
        plt.legend()
        
        fig_path = os.path.join(output_dir, f'training_history_{metric}.png')
        plt.savefig(fig_path, dpi=300)
        print(f"    - Saved {metric} plot to {fig_path}")
        plt.close()

def run_training_pipeline(data_path: str, output_dir: str, time_steps: int = 24):
    """Executes the full training pipeline."""
    print(f"🚀 STARTING AIMRAS V12.8 (REGULARIZED DEEP) TRAINING PIPELINE 🚀")
    os.makedirs(output_dir, exist_ok=True)

    # 1. Load and process data
    df = load_and_clean_data(data_path)
    df, target_labels = define_risk_labels(df)
    
    feature_cols = df.select_dtypes(include=np.number).columns.drop(target_labels, errors='ignore').tolist()
    
    # 2. Scale features and prepare data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[feature_cols])
    y = df[target_labels].values
    
    # 3. Create sequences
    X_seq, y_seq = create_sequences(X_scaled, y, time_steps)
    if len(X_seq) == 0:
        print(f"❌ ERROR: Not enough data to create sequences with {time_steps} steps.")
        return

    # 4. Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_seq, y_seq, test_size=0.2, random_state=SEED, shuffle=False
    )
    print(f"Train/Test split complete. Original train shape: {X_train.shape}")

    # 5. Apply SMOTE using the robust combination-string method
    print("--> Applying SMOTE to balance the training data...")
    n_samples, n_timesteps, n_features = X_train.shape
    X_train_reshaped = X_train.reshape((n_samples, n_timesteps * n_features))
    
    y_combined = np.array([''.join(row.astype(str)) for row in y_train])
    unique_classes, class_counts = np.unique(y_combined, return_counts=True)
    
    if len(unique_classes) > 1:
        min_samples = min(class_counts)
        k_neighbors = min(5, min_samples - 1) if min_samples > 1 else 1
        if k_neighbors >= 1:
            smote = SMOTE(random_state=SEED, k_neighbors=k_neighbors)
            X_train_smote_flat, y_combined_smote = smote.fit_resample(X_train_reshaped, y_combined)
            y_train_smote = np.array([list(label_str) for label_str in y_combined_smote], dtype=int)
            X_train_smote = X_train_smote_flat.reshape((-1, n_timesteps, n_features))
            print(f"--> SMOTE complete. New balanced train shape: {X_train_smote.shape}")
        else:
            print("--> SMOTE skipped. Not enough samples in the minority class.")
            X_train_smote, y_train_smote = X_train, y_train
    else:
        print("--> SMOTE skipped. Only one class combination present in training data.")
        X_train_smote, y_train_smote = X_train, y_train

    # 6. Build and train the model
    model = build_conv_lstm_model(input_shape=(X_train_smote.shape[1], X_train_smote.shape[2]), num_labels=len(target_labels))
    
    early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    
    history = model.fit(
        X_train_smote, y_train_smote,
        epochs=50,
        batch_size=128,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # 7. Evaluate the final model
    final_metrics = model.evaluate(X_test, y_test, return_dict=True)
    print("\n--- Final Model Evaluation on Test Set ---")
    for name, value in final_metrics.items():
        print(f"{name.capitalize():<12}: {value:.4f}")

    # 8. Generate and save training graphs
    plot_and_save_history(history, output_dir)

    # 9. Save the complete bundle
    print("\n--> Saving model and artifacts...")
    model.save(os.path.join(output_dir, 'aimras_lstm_model.keras'))
    
    bundle = {
        'scaler': scaler,
        'feature_names': feature_cols,
        'target_labels': target_labels,
        'time_steps': time_steps
    }
    joblib.dump(bundle, os.path.join(output_dir, 'aimras_lstm_bundle.pkl'))
    
    print(f"🎉 TRAINING COMPLETE! Artifacts saved to '{output_dir}'.")

# ------------------------------------------------------------------------------
# SECTION D: EXECUTION
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    DATA_FILE_PATH = '/kaggle/input/iot-indoor-air-quality/IoT_Indoor_Air_Quality_Dataset.csv'
    OUTPUT_DIRECTORY = 'aimras_outputs_v13'
    
    TIME_STEPS = 24
    
    run_training_pipeline(DATA_FILE_PATH, OUTPUT_DIRECTORY, TIME_STEPS)

2025-07-07 05:02:04.009169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751864524.031900     107 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751864524.038733     107 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751864524.058102     107 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751864524.058121     107 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751864524.058124     107 computation_placer.cc:177] computation placer alr

🚀 STARTING AIMRAS V12.8 (REGULARIZED DEEP) TRAINING PIPELINE 🚀
--> Loading and Cleaning Data...
Utilizing all 10 numeric columns: ['temperature', 'humidity', 'co2', 'pm2.5', 'pm10', 'tvoc', 'co', 'light_intensity', 'motion_detected', 'occupancy_count']
--> Defining Risk Labels...
Found 4 trainable labels: ['Risk_High_PM2.5', 'Comfort_Humidity_High', 'Comfort_Thermal_Hot', 'Comfort_Thermal_Cold']
--> Creating sequences with 24 time steps...
Train/Test split complete. Original train shape: (77947, 24, 14)
--> Applying SMOTE to balance the training data...
--> SMOTE complete. New balanced train shape: (327408, 24, 14)
--> Building Deeper, Regularized Conv-LSTM model...


I0000 00:00:1751864540.741844     107 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 14)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 24, 64)              │           2,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 24, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 24, 64)              │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 12, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 12, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 12, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 6, 256)              │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

...

 Trainable params: 525,892 (2.01 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/50


I0000 00:00:1751864553.339058     145 cuda_dnn.cc:529] Loaded cuDNN version 90300


2558/2558 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - auc: 0.6353 - loss: 0.7262 - precision: 0.5597 - recall: 0.3590 - val_auc: 0.7113 - val_loss: 0.5387 - val_precision: 0.5139 - val_recall: 0.5537 - learning_rate: 5.0000e-04
Epoch 2/50
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - auc: 0.6797 - loss: 0.6447 - precision: 0.5818 - recall: 0.4517 - val_auc: 0.7297 - val_loss: 0.5097 - val_precision: 0.5484 - val_recall: 0.5666 - learning_rate: 5.0000e-04
Epoch 3/50
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - auc: 0.7118 - loss: 0.6198 - precision: 0.5951 - recall: 0.5374 - val_auc: 0.7267 - val_loss: 0.5156 - val_precision: 0.5419 - val_recall: 0.5663 - learning_rate: 5.0000e-04
Epoch 4/50
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - auc: 0.7292 - loss: 0.6066 - precision: 0.6059 - recall: 0.5658 - val_auc: 0.7325 - val_loss: 0.5050 - val_precision: 0.5541 - val_recall: 0.5715 - learning_rate: 5.0000e-04
Epoch 5/50
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - auc: 0.7391 - loss: 0.599